In [100]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np

pd.set_option('display.max_rows', 500)

df = pd.read_csv('1980_2020.csv')
df = df.sort_index(ascending=False)

# Don't Need This Estimated Gross Boolean Column
df = df.drop('Estimated', axis=1)

print(df.shape)
print(df.dtypes)
df.head(2)

(19750, 11)
Rank             int64
Release         object
Genre           object
Budget          object
Running Time    object
Gross           object
Theaters        object
Total Gross     object
Release Date    object
Distributor     object
Release Year     int64
dtype: object


,Rank,Release,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Release Year
19749,455,Love Goes Public,-,-,-,$38,-,"$5,738",Oct 25,Aspiration Entertainment,2020
19748,454,Benjamin the Elephant (2020),-,-,-,$49,1,$49,Oct 16,Viva Pictures,2020


In [101]:
# Let's MErge The Year and Date and Convert To DateTime To Extract The Realse Quarter As A Feature
df['Release Date'] = df['Release Date'].str.replace('Feb 29', 'Feb 28')

df['Release Year'] = df['Release Year'].astype(str)

df['Release Date'] = df['Release Date'] + ' ' + df['Release Year']
df.head()

,Rank,Release,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Release Year
19749,455,Love Goes Public,-,-,-,$38,-,"$5,738",Oct 25 2020,Aspiration Entertainment,2020
19748,454,Benjamin the Elephant (2020),-,-,-,$49,1,$49,Oct 16 2020,Viva Pictures,2020
19747,453,Shooting the Mafia,-,-,-,$88,3,"$10,881",Nov 22 2020,Cohen Media Group,2020
19746,452,Chained for Life,-,-,-,$115,3,"$17,431",Sep 13 2020,Kino International,2020
19745,451,Asako I & II,-,-,-,$231,3,"$25,559",May 17 2020,Grasshopper Film,2020


In [102]:
#Convert To DateTime and Extract Quarter
df['Release Date'] = pd.to_datetime(df['Release Date'])

df['Quarter'] = df['Release Date'].dt.quarter

df.dtypes

Rank                     int64
Release                 object
Genre                   object
Budget                  object
Running Time            object
Gross                   object
Theaters                object
Total Gross             object
Release Date    datetime64[ns]
Distributor             object
Release Year            object
Quarter                  int64
dtype: object

In [107]:
# Let's Fix Up Some Of The str dtypes for Conversion to Int/Floats
df['Gross'] = df['Gross'].str.replace('$', '')
df['Gross'] = df['Gross'].str.replace(',', '')
df['Total Gross'] = df['Total Gross'].str.replace('$', '').str.replace(',', '')

# And Missing Values To Avoid Errors At Tghe Model Stage
df['Theaters'] = df['Theaters'].replace(',', '').replace('-', np.nan).astype(float)


# Too Many Categories To Attempt To Create A Feature Here
# df['Distributor'] = df['Distributor'].astype('category')

In [108]:
df[['Gross', 'Total Gross']] = df[['Gross', 'Total Gross']].astype(float)

In [109]:
# Setting Prediction Boundries/Conditions
conditions = [ 
(df['Total Gross'] <10000000),
(df['Total Gross'] >10000000) & (df['Total Gross'] <20000000),
(df['Total Gross'] >20000000) & (df['Total Gross'] <30000000),
(df['Total Gross'] >30000000) & (df['Total Gross'] <40000000),
(df['Total Gross'] >40000000) & (df['Total Gross'] <50000000),
(df['Total Gross'] >50000000) & (df['Total Gross'] <60000000),
(df['Total Gross'] >60000000) & (df['Total Gross'] <70000000),
(df['Total Gross'] >70000000) & (df['Total Gross'] <80000000),
(df['Total Gross'] >80000000)
]

categories = [0, 1, 2, 3, 4, 5, 6, 7, 8]

df['Total Gross Categories'] = np.select(conditions, categories)

In [110]:
#Lets Filla NA Screens as this may be an important feature
df['Theaters'].agg([np.mean, np.median])

mean      782.232847
median     74.000000
Name: Theaters, dtype: float64

In [122]:
# Lets Use Loc To Fillna Theater Counts With Anything Below 1 as Median (74)
# And Anything Above As Mean (782)
#t1 = (df['Total Gross Categories'] >= 1)
#t2 = (df['Total Gross Categories'] < 1) 

#df.loc[t1,'Theaters'] = df.loc[t1,'Theaters'].fillna(df.loc[t1,'Theaters'].mean())
#df.loc[t2,'Theaters'] = df.loc[t2,'Theaters'].fillna(df.loc[t2,'Theaters'].median())

In [123]:
#Loading The Distributor Market Share DF (Scraped From the-numbers) and Search For Label Matches
dist = pd.read_csv('/Users/darragh/Python/Selenium & Sentiment/Selenium, Vault ai Scraper/distributors.csv')
dist[dist['Distributor'].str.contains('Fox')]

,Rank,Distributor,Movies,Total Box Office,Tickets,Share
4,5,20th Century Fox,522,"$25,855,984,961",3792174776,11.02%
11,12,Fox Searchlight,227,"$2,608,358,365",364763390,1.11%
143,144,Fox International,8,"$9,117,748",1085608,0.00%
352,353,Blue Fox Entertainment,9,"$703,159",76823,0.00%
626,627,B.D. Fox Independent,1,"$41,914",5285,0.00%
802,803,Fox Meadow,1,"$8,158",1185,0.00%


In [124]:
# Appending Extra Distributors To dist Who Are Not Listed In dist But Have A Marketshare Above 0.0% in 2020 
dist = dist.append({'Rank':999,'Distributor':'IMAX',
              'Share':'3.7%'}, ignore_index=True)
dist = dist.append({'Rank':1000,'Distributor':'Screen Gems',
              'Share':'0.5%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Bleecker Street Media',
              'Share':'0.1%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Pantelion Films',
              'Share':'0.1%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Atlas Distribution Company',
              'Share':'0.05%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Dragon Dynasty',
              'Share':'0.1%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Affirm Films',
              'Share':'0.1%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Fox Atomic',
              'Share':'0.1%'}, ignore_index =True)
dist = dist.append({'Rank':1000,'Distributor':'Crown',
              'Share':'0.1%'}, ignore_index =True)

In [125]:
# Matching/Merging Multiplie df Labels To The dist List 
df.loc[df['Distributor'].str.contains('Disney'), 'Distributor'] = 'Walt Disney'
df.loc[df['Distributor'].str.contains('Century Fox'), 'Distributor'] = '20th Century Fox'
df.loc[df['Distributor'].str.contains('Metro-Goldwyn-Mayer'), 'Distributor'] = 'MGM'
df.loc[df['Distributor'].str.contains('Columbia Pictures'), 'Distributor'] = 'Columbia'
df.loc[df['Distributor'].str.contains('Universal Pictures'), 'Distributor'] = 'Universal'
df.loc[df['Distributor'].str.contains('Fox Searchlight'), 'Distributor'] = 'Fox Searchlight'
df.loc[df['Distributor'].str.contains('TriStar'), 'Distributor'] = 'Sony/TriStar'
df.loc[df['Distributor'].str.contains('Open Road Films'), 'Distributor'] = 'Open Road'
df.loc[df['Distributor'].str.contains('Samuel Goldwyn'), 'Distributor'] = 'Samuel Goldwyn Films'
df.loc[df['Distributor'].str.contains('RCR'), 'Distributor'] = 'RCR Media Group'
df.loc[df['Distributor'].str.contains('DreamWorks'), 'Distributor'] = 'Dreamworks SKG'

# Matching/Merging Single df Labels to the dist List
df['Distributor'] = df['Distributor'].str.replace('Atlantic Releasing Corporation', 'Atlantic')
df['Distributor'] = df['Distributor'].str.replace('Dimension Films', 'Weinstein/Dimension')
df['Distributor'] = df['Distributor'].str.replace('New Line Cinema', 'New Line')
df['Distributor'] = df['Distributor'].str.replace('Relativity Media', 'Relativity')
df['Distributor'] = df['Distributor'].str.replace('Oscilloscope', 'Oscilloscope Pictures')
df['Distributor'] = df['Distributor'].str.replace('Abramorama', 'Abramorama Films')
df['Distributor'] = df['Distributor'].str.replace('Cinema Libre Studio', 'Cinema Libre')
df['Distributor'] = df['Distributor'].str.replace('UTV Motion Pictures', 'UTV Communications')
df['Distributor'] = df['Distributor'].str.replace('The Weinstein Company', 'Weinstein Co.')
df['Distributor'] = df['Distributor'].str.replace('The Film Arcade', 'Film Arcade')
df['Distributor'] = df['Distributor'].str.replace('The Film Arcade', 'Film Arcade')
df['Distributor'] = df['Distributor'].str.replace('Eros International', 'Eros Entertainment')
df['Distributor'] = df['Distributor'].str.replace('Menemsha Films', 'Menemsha Entertainment')
df['Distributor'] = df['Distributor'].str.replace('Strand Releasing', 'Strand')
df['Distributor'] = df['Distributor'].str.replace('ArtAffects Entertainment', 'ArtAffects')
df['Distributor'] = df['Distributor'].str.replace('United Artists Releasing', 'United Artists')
df['Distributor'] = df['Distributor'].str.replace('Well Go USA Entertainment', 'Well Go USA')
df['Distributor'] = df['Distributor'].str.replace('Greenwich Entertainment', 'Greenwich') 
df['Distributor'] = df['Distributor'].str.replace('Zeitgeist Films', 'Zeitgeist')
df['Distributor'] = df['Distributor'].str.replace('The Cinema Guild', 'Cinema Guild')
df['Distributor'] = df['Distributor'].str.replace('Embassy Pictures', 'Embassy')
df['Distributor'] = df['Distributor'].str.replace('FUNimation Entertainment', 'FUNimation')
df['Distributor'] = df['Distributor'].str.replace('Sony Pictures Releasing', 'Sony Pictures')

In [126]:
# Mapping A Dictionary Of Distributors/Market Share To The df
dist_list = dist['Distributor'].tolist()
market = dist['Share'].tolist()

dist_share_dict = dict(zip(dist_list, market))
df['Market_Share'] = df['Distributor'].map(dist_share_dict)
df['Market_Share'] = df['Market_Share'].str.replace('%', '').astype(float)

In [142]:
#Filling The Rest of The Indie Distributors With 0%
df['Market_Share'] = df['Market_Share'].fillna('0.00')
df['Market_Share'] = df['Market_Share'].astype(float)
print(df.shape)
df.head(10)

(19750, 14)


,Rank,Release,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Release Year,Quarter,Total Gross Categories,Market_Share
19749,455,Love Goes Public,-,-,-,38.0,17.0,5738.0,2020-10-25,Aspiration Entertainment,2020,4,0,0.00
19748,454,Benjamin the Elephant (2020),-,-,-,49.0,1.0,49.0,2020-10-16,Viva Pictures,2020,4,0,0.00
19747,453,Shooting the Mafia,-,-,-,88.0,3.0,10881.0,2020-11-22,Cohen Media Group,2020,4,0,0.01
19746,452,Chained for Life,-,-,-,115.0,3.0,17431.0,2020-09-13,Kino International,2020,3,0,0.01
19745,451,Asako I & II,-,-,-,231.0,3.0,25559.0,2020-05-17,Grasshopper Film,2020,2,0,0.00
19744,450,It's for Your Own Good,-,-,-,256.0,1.0,256.0,2020-11-27,Distrib Films,2020,4,0,0.00
19743,449,The Load,-,-,-,292.0,2.0,28410.0,2020-08-30,Grasshopper Film,2020,3,0,0.00
19742,448,I Do Not Care If We Go Down in History as Barb...,-,-,-,293.0,2.0,11307.0,2020-07-19,Big World Pictures,2020,3,0,0.00
19741,447,Sol,-,-,-,327.0,1.0,327.0,2020-11-20,Distrib Films,2020,4,0,0.00
19740,446,Island of Lemurs: Madagascar,-,-,-,374.0,302.0,11272213.0,2020-04-04,Warner Bros.,2020,2,1,15.21


In [144]:
# Dtype Check
print(df.dtypes)
# Quick Conversion Of Release Year
df['Release Year'] = df['Release Year'].astype(int)

# In Future We May Scrape Running Times, Genre and Budget from BOM or IMDb But For Now Let's Save

Rank                               int64
Release                           object
Genre                             object
Budget                            object
Running Time                      object
Gross                            float64
Theaters                         float64
Total Gross                      float64
Release Date              datetime64[ns]
Distributor                       object
Release Year                       int64
Quarter                            int64
Total Gross Categories             int64
Market_Share                     float64
dtype: object


In [145]:
df.to_csv('Cleaned_1980_2020.csv')

# Predicting Box Office Return Using No Financial Data and XGBClassifier

In [146]:
# Create arrays for the features and the target: X, y
X, y = df[['Theaters', 'Quarter', 'Release Year', 'Market_Share']], df['Total Gross Categories']

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, stratify = y, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='multi:softmax', num_class = 9, n_estimators=50, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.772152


# Digging Into The Results
Taking a closer looks at the results shows theyre not great a predicting box office tentpoles (biggest earners)

In [168]:
# Counting the categories occurances from train_test_split's stratifying 
unique, counts = np.unique(preds, return_counts=True)
dict(zip(unique, counts))

{0: 2938, 1: 454, 2: 55, 3: 72, 4: 2, 6: 2, 7: 1, 8: 426}

In [150]:
from sklearn.metrics import confusion_matrix

#Let's take a look at where the error occured
confusion_matrix(y_test, preds, labels=[0,1,2,3,4,5,6,7 ])

array([[2670,   65,    2,    1,    0,    0,    0,    0],
       [ 141,  121,   15,   11,    0,    0,    0,    0],
       [  54,   80,   12,   14,    0,    0,    1,    0],
       [  25,   76,    7,   12,    0,    0,    0,    0],
       [  22,   35,    7,   15,    0,    0,    1,    1],
       [  12,   29,    6,    4,    0,    0,    0,    0],
       [   4,   18,    1,    6,    0,    0,    0,    0],
       [   0,   13,    2,    1,    2,    0,    0,    0]])

# RandomSearchCV
Let's try to eek out a little more accuracy with RandomSearchCV and a param_grid

In [148]:
from sklearn.model_selection import RandomizedSearchCV

# Define a parameter grid
rs_param_grid = {
    'max_depth': np.arange(30,50,5),
    'alpha': [0, 0.001, 0.01, 0.1],
    'subsample': [0.75],
    'learning_rate': np.linspace(00.1, 0.5, 10),
    'n_estimators': [100] }


# Insantiate XGBoost Clasifier 
xgb_clf = xgb.XGBClassifier(random_state=123)

# Instantiate RandomizedSearchCV()
xgb_rs = RandomizedSearchCV(estimator = xgb_clf, param_distributions = rs_param_grid, cv=3,
                            n_iter=5, verbose=1, n_jobs=3)

# Train the model on the training set
xgb_rs.fit(X_train, y_train)

# Print the best parameters and lowest RMSE
print(xgb_rs.best_params_)
print(xgb_rs.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.0min finished


{'subsample': 0.75, 'n_estimators': 100, 'max_depth': 40, 'learning_rate': 0.5, 'alpha': 0}
0.7966459766052009


# Almost 3% Higher, Not Bad
I will categorize the ditributors into groups to reduce the feature variance, then scrape the Budget, Genre and Running Times and return to this project in the future. I may then also be able to narrow the box office gross categories. Modest results but a fun a project nonetheless!